In [13]:
import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Dropout, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16

In [6]:
image_size = (224, 224)
num_classes = 300
train_batch_size = 64
val_batch_size = 32

In [7]:
def palm_model():
    x1 = Input(shape=(224, 224, 3),name='palm_input')
    x = Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu")(x1)
    x = Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu")(x)
    x = MaxPool2D(pool_size=(2,2),strides=(2,2))(x)
    x = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(x)
    x = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(x)
    x = MaxPool2D(pool_size=(2,2),strides=(2,2))(x)
    x = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(x)
    x = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(x)
    x = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(x)
    x = MaxPool2D(pool_size=(2,2),strides=(2,2))(x)
    x = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(x)
    x = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(x)
    x = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(x)
    x = MaxPool2D(pool_size=(2,2),strides=(2,2))(x)
    x = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(x)
    x = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(x)
    x = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(x)
    x = MaxPool2D(pool_size=(2,2),strides=(2,2),name='palm_vgg16')(x)
    x = Dropout(0.5)(x)
    x = Flatten(name='palm_flatten')(x)
    x = Dense(256, activation='relu', name='palm_fc1')(x)
    x = Dense(128, activation='relu', name='palm_fc2')(x)
    x = Dense(20, activation='softmax',name='palm_output')(x)
    return Model(inputs=x1, outputs=x)

In [8]:
def palm_model():
    model = VGG16(include_top=False, input_shape=(224, 224, 3))
    x = Flatten()(model.output)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(num_classes, activation='softmax')(x)
    return Model(model.inputs, x)

In [9]:
def read_palm_img(image_file):
    image = cv2.imread(image_file)
    image = cv2.resize(image, image_size)
    image = image * 1./255
    return image.astype(np.float32)

In [10]:
def generate_data(csv_file):
    def process():
        df = pd.read_csv(csv_file)
        df = df[['palm_print', 'label']]
        unique_labels = list(np.unique(df.label))
        num_labels = len(unique_labels)
        total_rows = df.shape[0]
        row_idx = 0
        while True:
            if row_idx % total_rows == 0:
                row_idx = 0
                df.sample(frac=1)  # Shuffle dataframe
            row = df.iloc[row_idx, :]
            row_idx += 1
            palm = read_palm_img(row['palm_print'])
            label = to_categorical(unique_labels.index(row['label']), num_classes=num_labels)
            yield palm, label
    return process

In [11]:
def data_generator(csv_file, batch_size):
    gen = generate_data(csv_file)
    dataset = tf.data.Dataset.from_generator(gen,
        output_types=(tf.float32, tf.float32),
        output_shapes=((224, 224, 3), num_classes)
    )
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(2)
    return dataset

In [15]:
lr = 0.0001
epochs = 5
num_train_samples = pd.read_csv('train.csv').shape[0]
num_val_samples = pd.read_csv('val.csv').shape[0]
steps_per_epoch = num_train_samples//train_batch_size
validation_steps = num_val_samples//val_batch_size
save_model_as = "palm_epochs{}_lr{}_batch{}"
model_output = save_model_as.format(epochs, lr, train_batch_size)

train_gen = data_generator('train.csv', batch_size=train_batch_size)
val_gen = data_generator('val.csv', batch_size=val_batch_size)
model = palm_model()
model.summary()
# model.compile(optimizer=Adam(lr=lr), loss='categorical_crossentropy', metrics=['accuracy'])
# history = model.fit(train_gen, epochs=epochs,
#                     steps_per_epoch=steps_per_epoch,
#                     validation_data=val_gen,
#                     validation_steps=validation_steps)
# model.save(model_output)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     